In [235]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from glob import glob
from PIL import Image
import os

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [236]:
data_dir = ''

In [237]:
# Verification of the number of Images 

# load filenames for human and dog images
dataset = np.array(glob(data_dir+"images/*"))
# print number of images in each dataset
print('There are %d total images.' % len(dataset))

There are 3642 total images.


In [238]:
# Check one image
#Image.open(open("images/Train_3.jpg", 'rb'))

In [239]:
sample_csv = pd.read_csv(data_dir + 'sample_submission.csv')
test_csv = pd.read_csv(data_dir + 'test.csv')
train_csv = pd.read_csv(data_dir + 'train.csv')


In [240]:
train_csv

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0
...,...,...,...,...,...
1816,Train_1816,0,0,0,1
1817,Train_1817,1,0,0,0
1818,Train_1818,1,0,0,0
1819,Train_1819,0,0,1,0


In [263]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, id_col, target_col, root_dir, sufix=None, transform=None):
        """
        Args:
            csv_file   (string):             Path to the csv file with annotations.
            root_dir   (string):             Directory with all the images.
            id_col     (string):             csv id column name.
            target_col (string):             csv target column name.
            sufix      (string, optional):   Optional sufix for samples.
            transform  (callable, optional): Optional transform to be applied on a sample.
        """
        self.data      = pd.read_csv(csv_file)
        self.id        = id_col
        self.target    = target_col
        self.root      = root_dir
        self.sufix     = sufix
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # get the image name at the different idx
        img_name = self.data.loc[idx, self.id]
        
        # if there is not sufic, nothing happened. in this case sufix is '.jpg'
        if self.sufix is not None:
            img_name = img_name + self.sufix
        
        # it opens the image of the img_name at the specific idx
        image = Image.open(os.path.join(self.root, img_name))
        
        # if there is not transform nothing happens, here we defined below two transforms for train and for test
        if self.transform is not None:
            image = self.transform(image)
        
        # define the label based on the idx
        #label = self.data.loc[idx, self.target].values
        #label = torch.from_numpy(label.astype(np.int8))
        #label = label.squeeze(-1)
        
        #Test second option
        
        label_test = self.data.iloc[idx, 1:5].values.astype('float32')
        
        return image, label_test

In [ ]:
class MyIterableDataset(torch.utils.data.IterableDataset):
...     def __init__(self, start, end):
...         super(MyIterableDataset).__init__()
...         assert end > start, "this example code only works with end >= start"
...         self.start = start
...         self.end = end
...
...     def __iter__(self):
...         worker_info = torch.utils.data.get_worker_info()
...         if worker_info is None:  # single-process data loading, return the full iterator
...             iter_start = self.start
...             iter_end = self.end
...         else:  # in a worker process
...             # split workload
...             per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
...             worker_id = worker_info.id
...             iter_start = self.start + worker_id * per_worker
...             iter_end = min(iter_start + per_worker, self.end)
...         return iter(range(iter_start, iter_end))
...
>>> # should give same set of data as range(3, 7), i.e., [3, 4, 5, 6].
>>> ds = MyIterableDataset(start=3, end=7)

>>> # Single-process loading
>>> print(list(torch.utils.data.DataLoader(ds, num_workers=0)))
[3, 4, 5, 6]

>>> # Mult-process loading with two worker processes
>>> # Worker 0 fetched [3, 4].  Worker 1 fetched [5, 6].
>>> print(list(torch.utils.data.DataLoader(ds, num_workers=2)))
[3, 5, 4, 6]

>>> # With even more workers
>>> print(list(torch.utils.data.DataLoader(ds, num_workers=20)))
[3, 4, 5, 6]

In [264]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

params = {
    'id_col':     'image_id',  
    'target_col': ['healthy', 'multiple_diseases', 'rust', 'scab'],
    'sufix':      '.jpg',
    'transform':  data_transforms['train']
}

train_dataset = CustomDataset(csv_file=data_dir+'train.csv', root_dir=data_dir+'images', **params)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


In [265]:
'''for idx, (data, target) in enumerate(train_loader):
    output = model_patho(data)
    print(data.shape)
    print(output.shape)
    print(target)
    print(target.shape)
'''

'for idx, (data, target) in enumerate(train_loader):\n    output = model_patho(data)\n    print(data.shape)\n    print(output.shape)\n    print(target)\n    print(target.shape)\n'

In [266]:
'''
for idx, (data, target) in enumerate(train_dataset):
    print(target)
    print(target.shape)
'''

'\nfor idx, (data, target) in enumerate(train_dataset):\n    print(target)\n    print(target.shape)\n'

In [267]:
'''
csv_file=data_dir+'train.csv'
img_name = pd.read_csv(csv_file).loc[1820, 'image_id']
        
# if there is not sufic, nothing happened. in this case sufix is '.jpg'
img_name = img_name + '.jpg'

# it opens the image of the img_name at the specific idx
root_dir=data_dir+'images'
image = Image.open(os.path.join(root_dir, img_name))

image_trans = data_transforms['train'](image)
        
# define the label based on the idx
labels = pd.read_csv(csv_file).loc[1820, ['healthy', 'multiple_diseases', 'rust', 'scab']].values
print('label from pd table: ',labels.shape)
labels = torch.from_numpy(labels.astype(np.int8))
print('Label moved to torch: ',labels.shape)
#label = label.squeeze(-1)
#print('Label unsquezzed: ', label.shape)

#img_name, image, image_trans, label.shape
#label.shape
'''

"\ncsv_file=data_dir+'train.csv'\nimg_name = pd.read_csv(csv_file).loc[1820, 'image_id']\n        \n# if there is not sufic, nothing happened. in this case sufix is '.jpg'\nimg_name = img_name + '.jpg'\n\n# it opens the image of the img_name at the specific idx\nroot_dir=data_dir+'images'\nimage = Image.open(os.path.join(root_dir, img_name))\n\nimage_trans = data_transforms['train'](image)\n        \n# define the label based on the idx\nlabels = pd.read_csv(csv_file).loc[1820, ['healthy', 'multiple_diseases', 'rust', 'scab']].values\nprint('label from pd table: ',labels.shape)\nlabels = torch.from_numpy(labels.astype(np.int8))\nprint('Label moved to torch: ',labels.shape)\n#label = label.squeeze(-1)\n#print('Label unsquezzed: ', label.shape)\n\n#img_name, image, image_trans, label.shape\n#label.shape\n"

In [268]:

'''
labels_Test = pd.read_csv(csv_file).loc[1820, ['healthy', 'multiple_diseases', 'rust', 'scab']].values
print(labels_Test.shape)
labels_Test = torch.from_numpy(labels_Test.astype(np.int8))
print(labels_Test.shape)
labels_Test = labels_Test.unsqueeze(-1)
print(labels_Test.shape)
'''

"\nlabels_Test = pd.read_csv(csv_file).loc[1820, ['healthy', 'multiple_diseases', 'rust', 'scab']].values\nprint(labels_Test.shape)\nlabels_Test = torch.from_numpy(labels_Test.astype(np.int8))\nprint(labels_Test.shape)\nlabels_Test = labels_Test.unsqueeze(-1)\nprint(labels_Test.shape)\n"

In [269]:
# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        # CL sees 224 x 224 x 3 image tensor
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # CL sees 112 x 112 x 16
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # CL sees 56 x 56 x 32
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # Max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 28 * 28 -> 500)
        self.fc1 = nn.Linear(64 * 28 * 28, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 4)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        ## Define forward behavior       
        x = self.pool(F.relu(self.conv1(x)))      
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))       
        # flatten image input
        x = x.view(-1, 64 * 28 * 28)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # output layer layer, with sigmoid activation function
        x = F.sigmoid(self.fc2(x))
        return x
    
    
model_patho = Net()
model_patho

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=4, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [285]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for idx, (data, target) in enumerate(loaders):

            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            #print(data)
            output = model(data)
            # calculate the batch loss
            #torch.max(target, 1)[1]
            #print('output shape:  ', output.shape)
            #target = target.view(-1)
            #print('target shape:  ', target.shape)
            #print('target type:  ', type(target))
            #print(target)
            
            label_inter = pd.read_csv(data_dir+'train.csv').iloc[idx, 1:5].values
            label_inter = torch.from_numpy(label_inter.astype(np.int64))
            label_inter = label_inter.squeeze(-1)
            label_inter = label_inter.view(-1)
            #label_inter = label_inter.Long()
            #print(label_inter)
            #print(label_inter.shape)
            #print(type(label_inter))
            #target = target.long()
            print(idx)
            loss = criterion(output, label_inter)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            #update training loss
            train_loss += loss.item()*data.size(0)
            
        # calculate average losses
        train_loss = train_loss/len(loaders.sampler)
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            ))
            
    # return trained model
    return model



In [286]:
### TODO: select loss function
criterion = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer = optim.SGD(model_patho.parameters(), lr=0.001)

In [287]:
# train the model
model_res = train(2, train_loader, model_patho, optimizer, criterion)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

ValueError: Expected input batch_size (1) to match target batch_size (4).